### NERC dataset citations - DataCite
Code to collect NERC dataset DOIs and citation information from the DataCite API

In [ ]:
import requests, time, json, re, datetime, os, sys
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import numpy as np
import pandas as pd
from math import ceil
from datetime import date

from citations_fun.getDataCiteCitations_relationTypes import getDataCiteCitations_relationTypes
from citations_fun.getPublicationInfo_forDataCite import getPublicationInfo


## DataCite events API
Get citation event data from datacite

In [ ]:
relation_type_id_list = ['is-cited-by', 'is-referenced-by', 'is-supplement-to', 'IsPartOf', 'IsContinuedBy', 'IsDescribedBy', 'IsDocumentedBy', 'IsDerivedFrom', 'IsRequiredBy']
dataCite_df_relationTypes = getDataCiteCitations_relationTypes(relation_type_id_list)

In [ ]:
# join events data with dataset DOI metadata

### don't need to load file if done in same env as getNERCDataDOIs() ###
with open("Results/intermediate_data/nerc_datacite_dois.json") as f:
    nerc_datacite_dois = json.load(f)

nerc_datacite_dois_df = pd.DataFrame(nerc_datacite_dois)

datacite_doi_events_df = dataCite_df_relationTypes.merge(
    nerc_datacite_dois_df,
    left_on='data_doi',
    right_on='data_doi',
    how='left'  # left-join as dataCite_df_relationTypes is a subset of nerc_datacite_dois_df
)

datacite_doi_events_df_drop = datacite_doi_events_df.drop(['data_page_number', 'data_self_link'], axis = 1)

# re-order
datacite_doi_events_df_drop = datacite_doi_events_df_drop[[
    'data_doi', 'data_publisher', 'data_title', 'data_publication_year', 'data_authors',
    'relation_type', 'pub_doi', 'source_id'
]]

In [ ]:
# takes 15 mins
# get pub title, authors and date
dataCite_df_pubInfo = getPublicationInfo(datacite_doi_events_df_drop)

In [ ]:
dataCite_df_pubInfo_names = dataCite_df_pubInfo[[
    'data_doi', 'data_publisher', 'data_title', 'data_publication_year', 'data_authors',
    'relation_type', 'pub_doi', 'pub_title', 'pub_date', 'pub_authors', 'source_id', 'pub_publisher', 'pub_type'
]]   

In [ ]:
dataCite_df_pubInfo_names.to_csv("Results/intermediate_data/latest_results_dataCite.csv", index= False)